In [0]:
%sql
create database scdtp2db;

In [0]:
%sql
use scdtp2db;

# create a target table

In [0]:
%sql
create or replace table targeted_table
using delta as
select * from values
(1, 'Satwanth', 'New York', '2025-01-05 00:00:00', '9999-09-09 00:00:00', 'Y'),
(2, 'Naidu', 'New Haven', '2025-01-04 00:00:00', '9999-09-09 00:00:00', 'Y')
as target(id, name, city, start_time, end_time, isActive)

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from targeted_table;

id,name,city,start_time,end_time,isActive
1,Satwanth,New York,2025-01-05 00:00:00,9999-09-09 00:00:00,Y
2,Naidu,New Haven,2025-01-04 00:00:00,9999-09-09 00:00:00,Y


#create source table

In [0]:
%sql
create or replace table staged_source
using delta as 
select * from values
(1, 'Satwanth', 'New York'),
(2, 'Naidu', 'Plano'),
(3, 'Var', 'Hartford')
as staged(id, name, city)

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from staged_source;

id,name,city
1,Satwanth,New York
2,Naidu,Plano
3,Var,Hartford


While getting new data from source, we need to compare it with existing data in our target and:
- If the record is same, keep it unchanged
- If its a new record, insert it with start_time = current_timestamp(), end_time with some hypothetical value and isActive as True
- If its an updated record, we expire existing active record by setting end_time = current_timestamp(), isActive to false and insert the enw record as active.

#Update old records

In [0]:
%sql
merge into targeted_table t
using staged_source s
on t.id = s.id 
when matched and (t.name <> s.name or t.city <> s.city) then
update set 
  t.end_time = current_timestamp(),
  t.isActive = 'N'


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,1,0,0


#Insert New and Updated records

In [0]:
%sql
insert into targeted_table 
select s.id, s.name, s.city, current_timestamp() as start_date, '9999-09-09 00:00:00' as end_date, 'Y' as isActive 
from staged_source s 
left join targeted_table t 
on s.id = t.id 
where t.id is null or (t.name <> s.name or t.city <> s.city)

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
select * from targeted_table;

id,name,city,start_time,end_time,isActive
2,Naidu,Plano,2025-12-01 15:29:12.925052,9999-09-09 00:00:00,Y
3,Var,Hartford,2025-12-01 15:29:12.925052,9999-09-09 00:00:00,Y
1,Satwanth,New York,2025-01-05 00:00:00,9999-09-09 00:00:00,Y
2,Naidu,New Haven,2025-01-04 00:00:00,2025-12-01 15:29:07.616102,N
